In [ ]:
import os
import random

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import f1_score
from tqdm import tqdm


PATH_DATA = "/home/a.makarchuk@rit.va/Desktop/kaggle-CV-best11/dogs_vs_cats/data/"
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 100
EPOCHS = 10
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
TRAIN_RATIO = 0.85
SEED = 42

In [ ]:
os.chdir(PATH_DATA)

submission_df = pd.read_csv(f"{PATH_DATA}sampleSubmission.csv")
list_train_imgs = os.listdir(f"{PATH_DATA}train")
list_test_imgs = os.listdir(f"{PATH_DATA}test1")

In [ ]:
# SHOW DATA

rand_img_train_path = f"{PATH_DATA}/train/{random.choice(list_train_imgs)}"
img = plt.imread(rand_img_train_path)
img = cv2.resize(img, IMAGE_SIZE, interpolation=cv2.INTER_AREA)

print(rand_img_train_path)

plt.imshow(img)
plt.show()

In [ ]:
def load_and_transform_img(path_img, image_size):
    img_name = path_img.split("/")[-1]
    label = 1 if img_name.split(".")[0] == "dog" else 0
    img = plt.imread(path_img)
    img = cv2.resize(img, image_size, interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32) / 255.0
    return torch.from_numpy(img).float(), torch.from_numpy(np.array(label)).float()


def generate_batch(list_imgs_path, batch_size):
    curr_id = 0
    while curr_id + batch_size < len(list_imgs_path):
        images_tensor_list = list()
        labels_tensor_list = list()
        for i in range(batch_size):
            img_tensor, label = load_and_transform_img(
                PATH_DATA + "train/" + list_imgs_path[curr_id + i], IMAGE_SIZE
            )
            images_tensor_list.append(img_tensor)
            labels_tensor_list.append(label)
        yield torch.stack(images_tensor_list), torch.stack(labels_tensor_list).unsqueeze(-1)
        curr_id += batch_size

In [ ]:
# CHECK LOADER


for i, (stacked_imgs, stacked_lbls) in enumerate(generate_batch(list_train_imgs, 1)):
    img = stacked_imgs.squeeze().numpy()
    label = stacked_lbls.squeeze().numpy()
    img = (img * 255).astype(int)
    print(label)
    plt.imshow(img)
    plt.show()
    if i > 5:
        break

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, image_size=IMAGE_SIZE):
        super().__init__()
        h, w = image_size
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(h * w * 3),
            nn.Linear(h * w * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
model = NeuralNetwork().to(DEVICE)
sigmoid = nn.Sigmoid()
opt = torch.optim.Adam(model.parameters(), lr=5e-3)
loss_fn = nn.BCELoss()

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
np.random.seed(SEED)
random.seed(SEED)

list_imgs = list_train_imgs[:]
random.shuffle(list_imgs)
list_train_imgs = list_imgs[: int(len(list_imgs) * TRAIN_RATIO)]
list_val_imgs = list_imgs[int(len(list_imgs) * TRAIN_RATIO) :]

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")

    model.train()
    for idx, (batch_imgs, labels) in tqdm(
        enumerate(generate_batch(list_train_imgs, BATCH_SIZE)),
        total=len(list_train_imgs) // BATCH_SIZE,
    ):
        batch_imgs = batch_imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        logits = model(batch_imgs)

        loss = loss_fn(sigmoid(logits), labels)
        loss.backward()
        opt.step()
        opt.zero_grad()

        predicted_probs = sigmoid(logits).detach().cpu().numpy()
        predicted_labels = (predicted_probs > 0.5).astype(float)

        if idx % 50 == 0:
            print(
                f"Train F1-score: {f1_score(labels.detach().cpu().numpy(), predicted_labels):.4f} Train Loss: {loss:.4f}"
            )

    model.eval()
    val_preds = []
    val_labels = []
    val_losses = []
    with torch.no_grad():
        for batch_imgs, labels in tqdm(
            generate_batch(list_val_imgs, BATCH_SIZE), total=len(list_val_imgs) // BATCH_SIZE
        ):
            batch_imgs = batch_imgs.to(DEVICE)
            labels = labels.to(DEVICE)

            logits = model(batch_imgs)
            val_losses.append(loss_fn(sigmoid(logits), labels).detach().cpu().numpy())
            predicted_probs = sigmoid(logits).detach().cpu().numpy()
            predicted_labels = (predicted_probs > 0.5).astype(float)

            val_preds.extend(predicted_labels)
            val_labels.extend(labels.detach().cpu().numpy())

    val_f1 = f1_score(val_labels, val_preds)
    print(f"Validation F1-score: {val_f1:.4f} Validation Loss: {np.mean(val_losses):.4f}")

In [ ]:
model.eval()
test_preds = pd.DataFrame(
    {
        "id": [path_img.split(".")[0] for path_img in list_test_imgs],
        "label": [
            sigmoid(
                model(
                    torch.from_numpy(
                        cv2.imread(f"{PATH_DATA}test1/{path_img}").astype(np.float32) / 255.0
                    ).unsqueeze(0)
                )
            ).item()
            for path_img in list_test_imgs
        ],
    }
)

In [ ]:
pd_test

In [ ]:
pd_test.to_csv("base_from_scratch.csv", index=False)